<a href="https://colab.research.google.com/github/moqri/deep_cell/blob/master/starter_code/intestin_cells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Download the data Atlas of human intestin data from 
# https://singlecell.broadinstitute.org/single_cell/study/SCP44
# and put in the /data folder
# in particular, you need the 
# 1. raw_expression_matrix.txt (for input)
# 2. metadata.txt (for labels)

In [11]:
# if you already have a compressed copy of the data (single_cell_portal_intestine.tar.gz) on Google Drive:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [12]:
compressed_file_path='/gdrive/My\ Drive/pr/cs230/data/single_cell_portal_intestin.tar.gz'
!mkdir data
!tar -xf {compressed_file_path} --directory /content/data/

mkdir: cannot create directory ‘data’: File exists


In [0]:
data_path='/content/data/'
folder='single_cell_portal_intestin/'
expression_file='atlas_Log2Tpm_round2.txt'
cell_file='atlas_metadata.txt'

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
import itertools

In [15]:
expression_df=pd.read_table(data_path+folder+expression_file,index_col=0).transpose()
expression_df.index.rename('Cell_ID',inplace=1)
expression_df.shape

(7216, 15971)

In [16]:
expression_df.head()

GENE,Mrpl15,Lypla1,Gm37988,Tcea1,Atp6v1h,Rb1cc1,4732440D04Rik,Fam150a,St18,Pcmtd1,Gm26901,Gm30414,Rrs1,Adhfe1,Mybl1,Vcpip1,1700034P13Rik,Sgk3,Mcmdc2,Snhg6,Tcf24,Cops5,Cspp1,Arfgef1,Slco5a1,Prdm14,Ncoa2,Tram1,Lactb2,Xkr9,Trpa1,Kcnb2,Terf1,Sbspon,Rpl7,Rdh10,Stau2,Ube2w,Tceb1,Tmem70,...,4930552P12Rik,Tcf7l2,Habp2,Nrap,Gm17197,Casp7,Plekhs1,Dclre1a,Nhlrc2,Adrb1,Ccdc186,Tdrd1,Vwa2,Afap1l2,Ablim1,B230217O12Rik,Fam160b1,Trub1,Atrnl1,Gfra1,Pnliprp1,Pnliprp2,Hspa12a,Eno4,Shtn1,Slc18a2,Pdzd8,Rps12-ps3,Rab11fip2,Fam204a,Cacul1,Nanos1,Eif3a,Fam45a,Sfxn4,Prdx3,Grk5,Zfp950,Gm7102,Csf2ra
Cell_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAACATTGTTTGGG_M1,2.10,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,3.64,0.0,0.0,0.0,1.40,1.40,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.00,0.0,0.0,2.56,0.0,0.0,0.0,0.0
AAACCGTGCCAAGT_M1,2.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.41,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
AAACGCACAGCCTA_M1,1.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.77,0.0,0.0,0.0,1.17,1.81,...,0.0,0.0,1.17,0.0,0.0,1.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.17,0.0,0.0,0.00,0.0,0.0,0.0,0.0
AAACGCACCAGATC_M1,1.71,1.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.42,0.0,0.0,0.0,1.71,0.00,...,0.0,0.0,0.00,0.0,0.0,2.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1.1,0.0,0.0,0.0,3.76,0.0,0.0,0.00,0.0,1.1,0.0,0.0
AAACGCACCGCCTT_M1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.3,0.0,0.0,0.0,0.0,0.0,4.61,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.00,1.3,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.3,0.0,0.0,1.97,0.0,0.0,1.30,0.0,0.0,0.0,0.0


In [17]:
cell_df=pd.read_table(data_path+folder+cell_file,skiprows=1,index_col=0)
cell_df.index.rename('Cell_ID',inplace=1)
cell_df=cell_df[['group.1']]
cell_df.columns=['Cell_Type']
cell_df.shape

(28723, 1)

In [18]:
cell_df.head()

,Cell_Type
Cell_ID,
AAACATTGTTTGGG_M1,Enterocyte.Immature.Distal
AAACCGTGCCAAGT_M1,Tuft
AAACGCACAGCCTA_M1,Goblet
AAACGCACCAGATC_M1,Enteroendocrine
AAACGCACCGCCTT_M1,Stem


In [19]:
cell_df['Cell_Type'].value_counts()

Stem                            6803
TA                              4164
Goblet                          2827
Enterocyte                      2499
Enterocyte.Progenitor           2178
TA.Early                        1970
EP                              1957
Tuft                             840
Enterocyte.Progenitor.Early      829
Paneth                           789
Enteroendocrine                  634
Enterocyte.Mature.Proximal       581
Enterocyte.Immature.Distal       512
TA.G2                            410
TA.G1                            408
Enterocyte.Progenitor.Late       404
Endocrine                        380
Enterocyte.Immature.Proximal     297
Enterocyte.Mature.Distal         241
Name: Cell_Type, dtype: int64

In [0]:
shared=set(cell_df.index)&set(expression_df.index)
expression_df=expression_df.loc[shared]
cell_df=cell_df.loc[shared]

In [0]:
cell_df['Cell_Type'] = pd.Categorical(cell_df['Cell_Type'])
cell_df['Cell_Type'] = cell_df['Cell_Type'].cat.codes

In [0]:
#expression_df_norm=(expression_df+1).transform(np.log)
expression_df_norm=np.round(expression_df,3)
selector = SelectKBest(chi2, k=1000)

In [23]:
selector.fit(expression_df, cell_df)
cols = selector.get_support(indices=True)
# Create new dataframe with only desired columns, or overwrite existing
expression_top_df = expression_df.iloc[:,cols]
expression_top_df.head()

GENE,St18,Trpa1,Ptpn18,Fhl2,Myo1b,Hspd1,Hspe1,Spats2l,Eef1b2,Cps1,Vil1,Cdk5r2,Fev,Cryba2,Ptprn,Resp18,Scg2,Itm2c,4933407L21Rik,Ncl,Ptma,Alpi,Inpp5d,Ramp1,Hes6,Ano7,Dtymk,Gal3st2,Pam,Tmem37,Lct,Mcm6,Nucks1,Btg2,Chil1,Ube2t,Rgs2,Rgs13,Pla2g4a,Glul,...,Hsbp1l1,Cndp2,Cyb5a,2010003K11Rik,Cst6,Slc25a45,Cdc42ep2,Naaladl1,Ehd1,Gm14964,Prdx5,Slc3a2,Eef1g,Ahnak,Incenp,Syt7,Tkfc,Ms4a10,Ms4a18,AW112010,Ms4a8a,Rfk,Aldh1a1,Gda,Asah2,Papss2,Ifit1bl2,Kif20b,Kif11,Rbp4,Hells,Cyp2c55,Cyp2c65,Cyp2c66,Cyp2c29,Cyp2c68,Sfrp5,Abcc2,Xpnpep1,Acsl5
Cell_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GATACTCTTAGCCA_M3,0.0,0.0,0.00,0.00,0.0,1.64,2.88,0.0,3.25,0.00,1.64,0.0,0.0,0.0,0.0,0.0,0.0,1.64,0.0,0.00,5.13,0.0,0.0,0.0,1.64,0.0,1.64,0.0,0.0,0.00,0.00,1.64,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.64,...,0.0,0.00,2.39,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,2.39,4.48,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.64,2.39,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,1.64,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00
TGCAACGATGGAAA_M6,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,2.97,0.00,4.17,0.0,0.0,0.0,0.0,0.0,0.0,2.14,0.0,0.00,1.71,1.1,0.0,0.0,0.00,0.0,1.10,0.0,0.0,2.14,1.10,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,3.87,3.49,0.0,2.47,3.16,1.71,0.00,0.0,0.0,2.14,2.14,1.71,0.00,0.00,0.0,2.47,2.97,1.71,1.10,3.49,3.87,0.0,1.10,0.0,1.1,1.71,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,2.47,2.47
CACTCCGATGGTCA_M1,0.0,0.0,0.00,0.00,0.0,2.67,3.55,0.0,2.67,2.67,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2.67,3.55,0.0,0.0,0.0,0.00,0.0,2.67,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,3.55,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,2.67,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,2.67,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00
AGCTCGCTACGGTT_M3,0.0,0.0,0.00,1.27,0.0,0.00,1.27,0.0,1.94,0.00,1.94,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1.27,0.00,0.0,0.0,0.0,0.00,0.0,1.27,0.0,0.0,1.27,1.27,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,3.45,2.39,0.0,2.74,0.00,0.00,2.74,0.0,0.0,1.94,1.94,2.74,0.00,1.27,0.0,0.00,1.27,3.01,0.00,2.74,1.27,0.0,1.27,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,1.27,0.0,0.0,1.94,0.0,0.0,3.25,3.01
TCAAGGACGCGAGA_M5,0.0,0.0,1.69,0.00,0.0,1.69,4.05,0.0,4.15,2.72,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2.45,4.67,0.0,0.0,0.0,0.00,0.0,3.73,0.0,0.0,0.00,0.00,1.69,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,3.31,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,4.15,1.08,0.00,0.0,0.00,1.08,0.00,1.08,0.00,1.08,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00


In [0]:
expression_array_top=expression_top_df.values
cell_array=cell_df.values
expression_array_normalized=np.round(expression_array_top) 

In [25]:
np.random.seed(1)
X_train, X_test, y_train, y_test = train_test_split(
    expression_array_normalized, cell_array, test_size=0.33, random_state=1)
clf = svm.SVC(decision_function_shape='ovo')
y_predict=clf.fit(X_train, y_train)
y_predict=clf.predict(X_test)
accuracy_score(y_test, y_predict)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7518891687657431

In [26]:
predictions=pd.DataFrame(data={'label':list(itertools.chain(*y_test)),'predicted':list(y_predict)})
predictions.head()

,label,predicted
0,10,10
1,5,5
2,5,5
3,10,10
4,5,5


In [27]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
tf.random.set_seed(1)

In [29]:
dataset=tf.data.Dataset.from_tensor_slices((expression_array_normalized,cell_array ))
dataset

<TensorSliceDataset shapes: ((1000,), (1,)), types: (tf.float64, tf.int8)>

In [30]:
for feat, targ in dataset.take(1):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [0. 0. 0. 0. 0. 2. 3. 0. 3. 0. 2. 0. 0. 0. 0. 0. 0. 2. 0. 0. 5. 0. 0. 0.
 2. 0. 2. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 2. 2. 0. 0. 2. 0. 0. 2. 0. 0. 2. 0. 2. 0. 0. 0. 0. 2. 0. 0. 2.
 2. 2. 0. 0. 0. 0. 0. 0. 2. 2. 2. 0. 2. 0. 0. 0. 2. 0. 0. 0. 0. 4. 0. 0.
 0. 0. 3. 0. 0. 2. 0. 3. 0. 0. 0. 2. 0. 2. 0. 2. 0. 0. 2. 0. 0. 0. 0. 0.
 3. 0. 2. 2. 0. 0. 0. 3. 0. 0. 0. 0. 2. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 4.
 0. 0. 0. 0. 2. 0. 3. 0. 0. 2. 0. 0. 2. 0. 0. 0. 0. 3. 2. 0. 0. 0. 0. 0.
 2. 0. 0. 0. 4. 2. 0. 2. 3. 2. 2. 2. 0. 0. 2. 2. 0. 0. 0. 0. 2. 0. 0. 0.
 0. 0. 0. 2. 2. 0. 4. 2. 0. 2. 0. 0. 0. 0. 0. 4. 0. 0. 0. 2. 0. 0. 0. 0.
 5. 0. 2. 0. 8. 0. 2. 2. 0. 0. 2. 0. 2. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 2. 0. 5. 0. 6. 5. 0. 0. 0. 2. 0. 0. 3. 2. 2. 0. 0. 0. 2. 0. 0. 0. 0.
 0. 0. 2. 0. 0. 0. 2. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 3. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 6. 0. 0. 0. 0. 0. 2. 0. 0. 2. 0. 0. 0.
 0. 0. 0. 3. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0

In [0]:
cell_type_count=len(cell_df['Cell_Type'].value_counts())

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(cell_type_count, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [45]:
model = get_compiled_model()
model.fit(X_train, y_train, epochs=10)
model.evaluate(X_test,  y_test, verbose=2)

Train on 4834 samples
Epoch 1/10
4834/4834 [==============================] - 0s 101us/sample - loss: 1.0637 - accuracy: 0.6359
Epoch 2/10
4834/4834 [==============================] - 0s 63us/sample - loss: 0.5523 - accuracy: 0.7933
Epoch 3/10
4834/4834 [==============================] - 0s 64us/sample - loss: 0.4050 - accuracy: 0.8558
Epoch 4/10
4834/4834 [==============================] - 0s 64us/sample - loss: 0.2926 - accuracy: 0.9048
Epoch 5/10
4834/4834 [==============================] - 0s 65us/sample - loss: 0.2197 - accuracy: 0.9363
Epoch 6/10
4834/4834 [==============================] - 0s 64us/sample - loss: 0.1649 - accuracy: 0.9586
Epoch 7/10
4834/4834 [==============================] - 0s 60us/sample - loss: 0.1160 - accuracy: 0.9772
Epoch 8/10
4834/4834 [==============================] - 0s 66us/sample - loss: 0.0923 - accuracy: 0.9826
Epoch 9/10
4834/4834 [==============================] - 0s 68us/sample - loss: 0.0647 - accuracy: 0.9923
Epoch 10/10
4834/4834 [=========

[0.92727497838106, 0.7325777]